<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtAlbert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers sentencePiece

     |████████████████████████████████| 1.8MB 13.6MB/s 
     |████████████████████████████████| 1.2MB 47.5MB/s 
     |████████████████████████████████| 890kB 44.0MB/s 
     |████████████████████████████████| 2.9MB 48.8MB/s 


In [2]:
import torch
from transformers import AlbertTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtAlbert Model<b>

In [3]:
tokenizer = AlbertTokenizer.from_pretrained("Rostlab/prot_albert", do_lower_case=False )

In [4]:
model = AutoModel.from_pretrained("Rostlab/prot_albert")

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0)

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Extracting sequences' features and covert the output to numpy if needed<b>

In [8]:
embedding = fe(sequences_Example)

In [9]:
embedding = np.array(embedding)

In [10]:
print(embedding)

[[[ 0.05823962  0.01667558 -0.07323031 ...  0.0572243   0.05712279
   -0.01743276]
  [ 0.05041254  0.01670932 -0.06041706 ...  0.03879357  0.06743018
   -0.02243913]
  [ 0.04481506  0.02007634 -0.06041673 ...  0.01449918  0.07357343
   -0.02993768]
  ...
  [ 0.03966146  0.04050564 -0.0189226  ...  0.02172126  0.06361968
   -0.04445013]
  [ 0.04467507  0.02889781 -0.06304034 ...  0.0083055   0.08290347
   -0.02485393]
  [ 0.05313296  0.0166304  -0.07059897 ...  0.04868817  0.06335299
   -0.01911608]]

 [[-0.03029598  0.01223707 -0.02814652 ... -0.12239455 -0.11794612
    0.06286692]
  [-0.0316638   0.01354288 -0.02900524 ... -0.11965024 -0.11535397
    0.0792664 ]
  [-0.04238674  0.01891162 -0.02534152 ... -0.12102054 -0.10938807
    0.09651565]
  ...
  [-0.03416523  0.01334407 -0.02930518 ... -0.12059835 -0.11478349
    0.0759825 ]
  [-0.04335058  0.02185565 -0.02340632 ... -0.12275703 -0.10738549
    0.10495753]
  [-0.04291698  0.02312784 -0.02060241 ... -0.12334881 -0.10543049
    0.

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtAlbert model<b>

In [11]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [12]:
print(features)

[array([[ 0.05041254,  0.01670932, -0.06041706, ...,  0.03879357,
         0.06743018, -0.02243913],
       [ 0.04481506,  0.02007634, -0.06041673, ...,  0.01449918,
         0.07357343, -0.02993768],
       [ 0.04433636,  0.02113897, -0.05988098, ...,  0.01485683,
         0.07245776, -0.02932404],
       ...,
       [ 0.04479509,  0.01773552, -0.06211109, ...,  0.01833772,
         0.07087235, -0.0290137 ],
       [ 0.03966146,  0.04050564, -0.0189226 , ...,  0.02172126,
         0.06361968, -0.04445013],
       [ 0.04467507,  0.02889781, -0.06304034, ...,  0.0083055 ,
         0.08290347, -0.02485393]]), array([[-0.0316638 ,  0.01354288, -0.02900524, ..., -0.11965024,
        -0.11535397,  0.0792664 ],
       [-0.04238674,  0.01891162, -0.02534152, ..., -0.12102054,
        -0.10938807,  0.09651565],
       [-0.04247037,  0.02128032, -0.02400274, ..., -0.12410947,
        -0.10866702,  0.10307093],
       [-0.04353855,  0.02157558, -0.02257719, ..., -0.12234777,
        -0.10860761,